In [16]:
import pandas as pd
import re

df = pd.read_csv("pt_question_answers.csv")

df

,pt_post_id,pt_post_type_id,pt_accepted_answer_id,pt_creation_date,pt_score,pt_title,pt_body,pt_tags,pt_parent_id,context,pt_answer,question
0,34750268,1,34762233.0,2016-01-12T17:36:25.473,9,Extracting the top-k value-indices from a 1-D Tensor,"<p>Given a 1-D tensor in Torch (<code>torch.Tensor</code>), containing value...",<python><lua><pytorch><torch>,NaN,<p>Just loop through the tensor and run your compare:</p>\n\n<pre><code>requ...,"<p>As of pull request <a href=""https://github.com/torch/torch7/pull/496"" rel...",Extracting the top-k value-indices from a 1-D Tensor <p>Given a 1-D tensor i...
1,38543850,1,38676842.0,2016-07-23T16:15:43.967,40,How to Display Custom Images in Tensorboard (e.g. Matplotlib Plots)?,"<p>The <a href=""https://github.com/tensorflow/tensorflow/blob/master/tensorf...",<python><tensorflow><matplotlib><pytorch><tensorboard>,NaN,"<p>It is quite easy to do if you have the image in a memory buffer. Below, I...","<p>It is quite easy to do if you have the image in a memory buffer. Below, I...",How to Display Custom Images in Tensorboard (e.g. Matplotlib Plots)? <p>The ...
2,41767005,1,43824857.0,2017-01-20T15:22:08.063,11,Python wheels: cp27mu not supported,"<p>I'm trying to install pytorch (<a href=""http://pytorch.org/"" rel=""norefer...",<python><linux><unicode><pytorch>,NaN,<p>This is exactly that. \nRecompile python under slack with --enable-unicod...,<p>This is exactly that. \nRecompile python under slack with --enable-unicod...,Python wheels: cp27mu not supported <p>I'm trying to install pytorch (<a hre...
3,41861354,1,54261158.0,2017-01-25T20:45:35.297,8,Loading Torch7 trained models (.t7) in PyTorch,"<p>I am using Torch7 library for implementing neural networks. Mostly, I re...",<python><lua><pytorch><torch><pre-trained-model>,NaN,<p>The correct function is <code>load_lua</code>:</p>\n\n<pre><code>from tor...,<p>As of PyTorch 1.0 <code>torch.utils.serialization</code> is completely re...,Loading Torch7 trained models (.t7) in PyTorch <p>I am using Torch7 library ...
4,41924453,1,42054194.0,2017-01-29T18:31:24.687,65,PyTorch: How to use DataLoaders for custom Datasets,<p>How to make use of the <code>torch.utils.data.Dataset</code> and <code>to...,<python><torch><pytorch>,NaN,"<p>Yes, that is possible. Just create the objects by yourself, e.g.</p>\n\n<...","<p>Yes, that is possible. Just create the objects by yourself, e.g.</p>\n\n<...",PyTorch: How to use DataLoaders for custom Datasets <p>How to make use of th...
...,...,...,...,...,...,...,...,...,...,...,...,...
10758,74612146,1,NaN,2022-11-29T09:54:30.430,0,Is it possible to perform quantization on densenet169 and how?,<p>I have been trying to performing quantization on a densenet model without...,<machine-learning><pytorch><artificial-intelligence><densenet><static-quanti...,NaN,"<p>Here's how to do this on DenseNet169 from torchvision:</p>\n<pre class=""l...","<p>Here's how to do this on DenseNet169 from torchvision:</p>\n<pre class=""l...",NaN
10759,74637151,1,NaN,2022-12-01T05:08:37.150,1,"Why when the batch size increased, the epoch time will also increasing?",<p>Epoch time means the time required to train for an epoch.</p>\n<p>From my...,<deep-learning><pytorch>,NaN,"<p>As you already noticed, there are many factors that may affect epoch-time...","<p>As you already noticed, there are many factors that may affect epoch-time...",NaN
10760,74642594,1,NaN,2022-12-01T13:23:27.277,0,Why does StableDiffusionPipeline return black images when generating multipl...,"<p>I am using the <a href=""https://github.com/huggingface/diffusers/tree/mai...",<python><pytorch><apple-m1><huggingface-transformers><stable-diffusion>,NaN,"<p>Apparently it is indeed an Apple Silicon (M1/M2) issue, of which Hugging ...","<p>Apparently it is indeed an Apple Silicon (M1/M2) issue, of which Hugging ...",NaN
10761,74671399,1,NaN,2022-12-03T22:46:46.443,1,Locating tags in a string in PHP (with respect to the string with tags removed),<p>I want to create a function that la

In [4]:



# df["text"] = "question: " + df["pt_title"] + "\n" + df["pt_body"] + "\n" + "answer: " + df["context"]
df["text"] = "question: " + df["pt_title"] + "\n" + "answer: " + df["pt_answer"]


df = df[["text"]]

CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

df["text"] = df["text"].apply(lambda x: cleanhtml(x))

df["text"] = df["text"].str.lower()


df

/tmp/ipykernel_1629/696361460.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].apply(lambda x: cleanhtml(x))
/tmp/ipykernel_1629/696361460.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.lower()


,text
0,question: extracting the top-k value-indices f...
1,question: how to display custom images in tens...
2,question: python wheels: cp27mu not supported\...
3,question: loading torch7 trained models (.t7) ...
4,question: pytorch: how to use dataloaders for ...
...,...
10758,question: is it possible to perform quantizati...
10759,"question: why when the batch size increased, t..."
10760,question: why does stablediffusionpipeline ret...
10761,question: locating tags in a string in php (wi...


In [5]:
from haystack import Document


# Use data to initialize Document objects
#titles = list(df["pt_title"].values)
#texts = list(df["pt_answer"].values)
texts = list(df["text"].values)
documents = []
#for title, text in zip(titles, texts):
for text in texts:
    documents.append(Document(content=text))


In [6]:
len(documents)

10763

In [7]:
documents[0]

<Document: {'content': 'question: extracting the top-k value-indices from a 1-d tensor\nanswer: as of pull request #496 torch now includes a built-in api named torch.topk. example:\n\n&gt; t = torch.tensor{9, 1, 8, 2, 7, 3, 6, 4, 5}\n\n-- obtain the 3 smallest elements\n&gt; res = t:topk(3)\n&gt; print(res)\n 1\n 2\n 3\n[torch.doubletensor of size 3]\n\n-- you can also get the indices in addition\n&gt; res, ind = t:topk(3)\n&gt; print(ind)\n 2\n 4\n 6\n[torch.longtensor of size 3]\n\n-- alternatively you can obtain the k largest elements as follow\n-- (see the api documentation for more details)\n&gt; res = t:topk(3, true)\n&gt; print(res)\n 9\n 8\n 7\n[torch.doubletensor of size 3]\n\n\nat the time of writing the cpu implementation follows a sort and narrow approach (there are plans to improve it in the future). that being said an optimized gpu implementation for cutorch is currently being reviewed.\n', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'e

In [8]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever,EmbeddingRetriever


# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = EmbeddingRetriever(document_store=document_store,
                              embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
                               model_format="sentence_transformers")

# Initialize DPR Retriever to encode documents, encode question and query documents
# retriever = DensePassageRetriever(
#     document_store=document_store,
#     query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#     passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#     use_gpu=True,
#     embed_title=True,
# )



/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It m

In [9]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


Writing Documents:   0%|          | 0/10763 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/10763 [00:00<?, ? docs/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

In [13]:
# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=512,
    min_length=10,
    embed_title=True,
    num_beams=10,
)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It m

In [11]:
top_10_questions = pd.read_csv("top100questions.csv").iloc[:10].question.tolist()

top_10_questions

['How do I check if PyTorch is using the GPU?\n',
 'How do I save a trained model in PyTorch?\n',
 'What does .view() do in PyTorch?\n',
 'Why do we need to call zero_grad() in PyTorch?\n',
 'How do I print the model summary in PyTorch?\n',
 'How do I initialize weights in PyTorch?\n',
 'What does model.eval() do in pytorch?\n',
 "What's the difference between reshape and view in pytorch?\n",
 'What does model.train() do in PyTorch?\n',
 'What does .contiguous() do in PyTorch?\n']

In [14]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in top_10_questions:
    res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 3}})
    print_answers(res, details="minimum")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: How do I check if PyTorch is using the GPU?

Answers:
[{'answer': ' import torch. is / is not a device'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: How do I save a trained model in PyTorch?

Answers:
[{'answer': ' fabulac Rabindra née saha'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


Query: What's the difference between reshape and view in pytorch?

Answers:
[{'answer': ' torch. reshape ( 0 : 1 )'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What does model.train() do in PyTorch?

Answers:
[{'answer': ' ⁄ 2 : model.train ( mode )'}]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What does .contiguous() do in PyTorch?

Answers:
[{'answer': ' notoriously paraphylicia pleasonton ( u.s. )'}]
